In [ ]:
!pip install -q patchify

In [ ]:
# import the necessary packages
import rasterio
import gc
import numpy as np
from patchify import patchify, unpatchify

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
class Config():
    def __init__(self):
        # Dataset/Dataloaders params
        self.years = ['2016','2017','2018','2019','2020','2021','2022']
        self.months = ['1','2','3','4', '11','12']
        self.base_dir = '../input/cdmipcamergeddataset/CDMI-PCA/CDMI_'
        self.save_path = './data_cube.npy'
        self.patch_size = 256
        
        self.seed = 26012022

In [ ]:
def read_rasters(base_dir, years, months):
    cdmi_paths = [base_dir+j+'_'+i+'.tif' for i in years for j in months][40]
    cdmi_stack = np.stack([rasterio.open(f).read() for f in cdmi_paths], axis=1)
    cdmi_stack[np.isnan(cdmi_stack)] = 0
    gc.collect()
    return cdmi_stack[:,:,:4352,:5120]  # (1, 40, 4352, 5120)

def patchify_images(cdmi_stack, patch_size, save_path):
    patchified_images = patchify(cdmi_stack, (1, 40, patch_size, patch_size), step=patch_size).squeeze()  #(17, 20, 128, 128))
    data_cube = patchified_images.reshape((-1, 40, patch_size, patch_size))
    np.save(save_path, data_cube)
    gc.collect()

In [ ]:
config = Config()
cdmi_stack = read_rasters(config.base_dir, config.years, config.months)
patchify_images(cdmi_stack, config.patch_size, config.save_path)